## Import necessary modules
Run this cell before running any other cells

In [2]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import time
import numpy as np

LOG.propagate = False

# Printing and Logging
## Printing
You can use the **print()** function in Python to print messages to the screen. <br>
The message can be a string, or any other object, the object will be converted into a string before it is written to the screen. <br>

## Logging
You could use the logging module that is setup in *utils.py*. <br>
It prints to both your screen (standard output) as well as to log files (*ble.log*) in the *logs* directory. <br>
This is the recommended way to output messages, since the log files can help with debugging. <br>
The logging module also provides different log levels as shown below, each formatted with a different color for increased visibility. <br>

__**NOTE**__: You may notice that the DEBUG message is not printed to the screen but is printed in the log file. This is because the logging level for the screen is set to INFO and for the file is set to DEBUG. You can change the default log levels in *utils.py* (**STREAM_LOG_LEVEL** and **FILE_LOG_LEVEL**). 

## Formatting output
To format your strings, you may use %-formatting, str.format() or f-strings. <br>
The most "pythonic" way would be to use f-strings. [Here](https://realpython.com/python-f-strings/) is a good tutorial on f-strings. <br>

In [3]:
LOG.debug("dd")
LOG.info("ifo")
LOG.warning("warning")
LOG.error("error")
LOG.critical("critical")

2022-02-08 21:24:36,476 | INFO     |: ifo
2022-02-08 21:24:36,477 | WARNING  |: warning
2022-02-08 21:24:36,477 | ERROR    |: error
2022-02-08 21:24:36,477 | CRITICAL |: critical


<hr>

# BLE
## ArtemisBLEController
The class **ArtemisBLEController** (defined in *ble.py*) provides member functions to handle various BLE operations to send and receive data to/from the Artemis board, provided the accompanying Arduino sketch is running on the Artemis board. <br>

<table align="left">
     <tr>
        <th style="text-align: left; font-size: medium">Member Functions</th>
        <th style="text-align: left; font-size: medium">Description</th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">reload_config()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Reload changes made in <em>connection.yaml.</em></span></th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">connect()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Connect to the Artemis board, whose MAC address is specified in <em>connection.yaml</em>.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">disconnect()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Disconnect from the Artemis board.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">is_connected()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Return a boolean indicating whether your controller is connected to the Artemis board or not.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">send_command(cmd_type, data)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Send the command <strong>cmd_type</strong> (integer) with <strong>data</strong> (string) to the Artemis board.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">receive_float(uuid) <br> receive_string(uuid) <br> receive_int(uuid)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Read the GATT characteristic (specified by its <strong>uuid</strong>) of type float, string or int. <br> The type of the GATT
            characteristic is determined by the classes BLEFloatCharacteristic, BLECStringCharacteristic or
            BLEIntCharacteristic in the Arduino sketch.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">start_notify(uuid, notification_handler)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Activate notifications on the GATT characteristic (specified by its <strong>uuid</strong>). <br> <strong>notification_handler</strong> is a
            function callback which must accept two inputs; the first will be a uuid string object and the second will
            be the bytearray of the characteristic value.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">bytearray_to_float(byte_array) <br> bytearray_to_string(byte_array) <br> bytearray_to_int(byte_array)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Convert the <strong>bytearray</strong> to float, string or int, respectively. <br> You may use these functions inside your
            notification callback function.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">stop_notify(uuid)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Stop notifications on the GATT characteristic (specified by its <strong>uuid</strong>).</span></th>
    </tr>
</table>

<table align="left">
     <tr>
        <th style="text-align: left; font-size: medium">Member Variables</th>
        <th style="text-align: left; font-size: medium">Description</th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">uuid</span></th>
        <th style="text-align: left"><span style="font-weight: normal">A dictionary that stores the UUIDs of the various characteristics specified in <em>connection.yaml</em>.</span></th>
    </tr>
</table>

## Configuration
- The MAC address, Service UUID and GATT characteristic UUIDs are defined in the file: *connection.yaml*.
- They should match the UUIDs used in the Arduino sketch.
- The artemis board running the base code should display its MAC address in the serial monitor.
- Update the **artemis_address** in *connection.yaml*, accordingly.
- Make sure to call **ble.reload_config()** or **get_ble_controller()** (which internally calls **reload_config()**) after making any changes to your configuration file.

<hr>

In the below cell, we create an **ArtemisBLEController** object using **get_ble_controller()** (defined in *ble.py*), which creates and/or returns a single instance of **ArtemisBLEController**. <br>
<span style="color:rgb(240,50,50)"> __NOTE__: Do not use the class directly to instantiate an object. </span><br>

In [43]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2022-02-10 02:08:41,756 | INFO     |: Looking for Artemis Nano Peripheral Device: c0:07:c1:8d:32:44
2022-02-10 02:08:44,227 | INFO     |: Connected to c0:07:c1:8d:32:44


## Receive data from the Artemis board

The cell below shows examples of reading different types (as defined in the Arduino sketch) of GATT characteristics.

In [6]:
# Read a float GATT Charactersistic
f = ble.receive_float(ble.uuid['RX_FLOAT'])
print(f)

15.5


In [7]:
# Read a string GATT Charactersistic
s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

[->9.0<-]


## Send a command to the Artemis board
Send the PING command and read the reply string from the string characteristic RX_STRING. <br>
__NOTE__: The **send_command()** essentially sends a string data to the GATT characteristic (TX_CMD_STRING). The GATT characteristic in the Arduino sketch is of type BLECStringCharacteristic.

In [15]:
ble.send_command(CMD.PING, "")

In [16]:
s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

PONG
2022-02-08 21:47:43,581 | INFO     |: Disconnected from 85F6AB81-3039-0348-6BFC-45DA0B5D18A8


The cell below shows an example of the SEND_TWO_INTS command. <br> The two values in the **data** are separated by a delimiter "|". <br>
Refer Lab 2 documentation for more information on the command protocol.

In [10]:
ble.send_command(CMD.SEND_TWO_INTS, "2|-6")

The Artemis board should print the two integers to the serial monitor in the ArduinoIDE. 

## Disconnect

In [15]:
# Disconnect
ble.disconnect()

2022-02-09 02:02:25,964 | INFO     |: Disconnected from 85F6AB81-3039-0348-6BFC-45DA0B5D18A8


## ECHO

In [24]:
ble.send_command(CMD.ECHO, "HiHello")

In [25]:
s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

Robot says -> HiHello :)


## Send three floats

In [34]:
ble.send_command(CMD.SEND_THREE_FLOATS, "1.1|2.2|3.3")

## Notify

In [44]:
f = 0.0
def update(raw,b):
    global f
    f = ble.bytearray_to_float(b)
    print(f)

ble.start_notify(ble.uuid['RX_FLOAT'], update)

4810.0
4810.5
4811.0
4811.5
4812.0
4812.5
4813.0
4813.5
4814.0
4814.5
4815.0
4815.5
4816.0
4816.5
4817.0
4817.5
4818.0
4818.5
4819.0
4819.5
4820.0
4820.5
4821.0
4821.5
4822.0
4822.5
4823.0
4823.5
4824.0
4824.5
4825.0
4825.5
4826.0
4826.5
4827.0
4827.5
4828.0
4828.5
4829.0
4829.5
4830.0
4830.5
4831.0
4831.5
4832.0
4832.5
4833.0
4833.5
4834.0
4834.5
4835.0
4835.5
4836.0
4836.5
4837.0
4837.5
4838.0
4838.5
4839.0
4839.5
4840.0
4840.5
4841.0
4841.5
4842.0
4842.5
4843.0
4843.5
4844.0
4844.5
4845.0
4845.5
4846.0
4846.5
4847.0
4847.5
4848.0
4848.5
4849.0
4849.5
4850.0
4850.5
4851.0
4851.5
4852.0
4852.5
4853.0
4853.5
4854.0
4854.5
4855.0
4855.5
4856.0
4856.5
4857.0
4857.5
4858.0
4858.5
4859.0
4859.5
4860.0
4860.5
4861.0
4861.5
4862.0
4862.5
4863.0
4863.5
4864.0
4864.5
4865.0
4865.5
4866.0
4866.5
4867.0
4867.5
4868.0
4868.5
4869.0
4869.5
4870.0
4870.5
4871.0
4871.5
4872.0
4872.5
4873.0
4873.5
4874.0
4874.5
4875.0
4875.5
4876.0
4876.5
4877.0
4877.5
4878.0
4878.5
4879.0
4879.5
4880.0
4880.5
4881.0

In [45]:
ble.stop_notify(ble.uuid['RX_FLOAT'])

5204.0
5204.5
5205.0
5205.5
5206.0
5206.5
5207.0
5207.5
5208.0
5208.5
5209.0
5209.5
5210.0
5210.5
5211.0
5211.5
5212.0
5212.5
5213.0
5213.5
5214.0
5214.5
2022-02-10 02:10:17,573 | INFO     |: Disconnected from 85F6AB81-3039-0348-6BFC-45DA0B5D18A8


## 5960 Task

## Effective Data Rate

In [20]:
ble.stop_notify(ble.uuid['RX_STRING'])

In [21]:
import time
t0 = 0.0
t1 = 0.0
T = 0.0
def update(raw,b):
    #print("updated")
    global T
    t1 = time.time()
    T += t1-t0
    
ble.start_notify(ble.uuid['RX_STRING'], update)

In [25]:
s = "aaaaa"
for i in range(1,30):
    #print(i)
    T = 0.0
    m = s*i
    for j in range(1000):
        ble.send_command(CMD.CAL1,m)
        t0 = time.time()
        
    print(T/1000)

0.015908139944076537
0.016672813892364503
0.01676095151901245
0.016696115493774415
0.01646072483062744
0.01620481204986572
0.016795154809951782
0.016531054496765136
0.016535114765167235
0.018204821825027466
0.018332424163818358
0.018144126176834106
0.017646334409713745
0.018550395250320435
0.01792791199684143
0.018816293478012086
0.017810901641845702
0.018045409202575684
0.018063746690750122
0.020378988265991212
0.020037018060684204
0.019557096242904665
0.018606719255447386
0.02032124900817871
0.02001492214202881
0.019573315858840944
0.01942130160331726
0.02016011881828308
0.019591639757156373


## Reliability

In [42]:
ble.stop_notify(ble.uuid['RX_STRING'])

In [40]:
import time
def update(raw,b):
    print("updated")
    
ble.start_notify(ble.uuid['RX_STRING'], update)

In [41]:
s = "11111"
for i in range(1,4):
    print("want to send " + str(i))
    ble.send_command(CMD.CAL2,s*i)
    print("sent " + str(i))

want to send 1
sent 1
want to send 2
updated
sent 2
want to send 3
updated
sent 3
updated


In [26]:
s = "aaaaa"
for i in range(1,30):
    #print(i)
    T = 0.0
    m = s*i
    for j in range(1000):
        ble.send_command(CMD.CAL1,m)
        t0 = time.time()
        
    print(T/1000)

0.016409653902053834
0.016371978521347044
0.016461201190948488
0.016122536182403564
0.01651046872138977
0.016577855110168457
0.016664616107940674
0.016961124897003174
0.017090125799179078
0.01766969966888428
0.017742019653320312
0.017551602125167848
0.01785804319381714
0.017924845933914184
0.018126641511917115
0.017913901329040527
0.018086018562316893
0.01796232581138611
0.01760334324836731
0.02013079023361206
0.01974570107460022
0.019629175901412964
0.018839187145233156
0.01879546856880188
0.019514275312423706
0.019646010637283325
0.01918830895423889
0.019627923488616944
0.01952640104293823


In [27]:
s = "aaaaa"
for i in range(1,30):
    #print(i)
    T = 0.0
    m = s*i
    for j in range(1000):
        ble.send_command(CMD.CAL1,m)
        t0 = time.time()
        
    print(T/1000)

0.015919650554656984
0.01640602231025696
0.01694652533531189
0.01685551905632019
0.01659947872161865
0.01644185709953308
0.017089930057525633
0.01692059135437012
0.015752571582794188
0.01760269021987915
0.017837212800979614
0.017584359169006348
0.017867415428161622
0.017588807582855224
0.017555493116378783
0.01769032049179077
0.01784418034553528
0.017961087703704835
0.0179171941280365
0.02030408716201782
0.020222527265548705
0.020290837526321413
0.020100001573562624
0.020101122856140137
0.01993686294555664
0.019334295988082884
0.01957124948501587
0.0193000750541687
0.019495480060577393
